<a href="https://colab.research.google.com/github/soham-slp/Tf-from-scratch/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Tensorflow variables for linear regression

In [1]:
import numpy as np
from plotly import express as px
from matplotlib import pyplot as plt
import tensorflow as tf

In [11]:
from plotly import graph_objects as go

In [2]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [110]:
xs = np.arange(-100.0, 100.0, 1.0)
ys = 2 * xs - 1 + np.random.normal(size = xs.shape) * 0.1

In [111]:
fig = px.line(x = xs, y = ys, title = 'y = 2x - 1', markers = True)
fig.update_layout({
    'showlegend': True,
    'xaxis_title': 'X',
    'yaxis_title': 'Y',
    'title': {
        'font': {
            'size': 50
        }
    },
    'font': {
        'family': 'Arial'
    }
})
fig.show()

In [168]:
X = tf.constant(xs)
X = tf.reshape(xs, shape = (1, 200))
Y = tf.constant(ys)
Y = tf.reshape(Y, shape = (1, 200))

In [169]:
@tf.function
def loss(y_true, y_pred):
  return 0.5 * tf.reduce_mean(tf.square(y_true - y_pred))

In [170]:
w = np.random.random(size = (Y.shape[0], X.shape[0])) * 0.001
bs = np.zeros((1, Y.shape[0]))

In [171]:
W = tf.Variable(initial_value = w, dtype = tf.float64, trainable = True)
b = tf.Variable(initial_value = bs, dtype = tf.float64, trainable = True)

In [172]:
@tf.function
def predict(X):
  return tf.matmul(W, X) + b

In [177]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.0001)
@tf.function
def train():
  with tf.GradientTape() as tape:
    Yhat = predict(X)
    cost = loss(Yhat, Y)
  grads = tape.gradient(cost, [W, b])
  optimizer.apply_gradients(zip(grads, [W, b]))

In [174]:
W

<tf.Variable 'Variable:0' shape=(1, 1) dtype=float64, numpy=array([[0.00096565]])>

In [175]:
train()

[<tf.Tensor 'PartitionedCall_2:1' shape=(1, 1) dtype=float64>, <tf.Tensor 'PartitionedCall_2:2' shape=(1, 1) dtype=float64>]
[<tf.Tensor 'PartitionedCall_2:1' shape=(1, 1) dtype=float64>, <tf.Tensor 'PartitionedCall_2:2' shape=(1, 1) dtype=float64>]


In [176]:
W

<tf.Variable 'Variable:0' shape=(1, 1) dtype=float64, numpy=array([[0.66733067]])>

In [184]:
for epoch in range(1, 10001):
  if epoch % 1000 == 0:
    print(f'After { epoch }:\nWeight: { W }\nBias: { b }\n\n')
  train()

After 1000:
Weight: <tf.Variable 'Variable:0' shape=(1, 1) dtype=float64, numpy=array([[1.99981222]])>
Bias: <tf.Variable 'Variable:0' shape=(1, 1) dtype=float64, numpy=array([[-0.99067565]])>


After 2000:
Weight: <tf.Variable 'Variable:0' shape=(1, 1) dtype=float64, numpy=array([[1.99981214]])>
Bias: <tf.Variable 'Variable:0' shape=(1, 1) dtype=float64, numpy=array([[-0.99119851]])>


After 3000:
Weight: <tf.Variable 'Variable:0' shape=(1, 1) dtype=float64, numpy=array([[1.99981207]])>
Bias: <tf.Variable 'Variable:0' shape=(1, 1) dtype=float64, numpy=array([[-0.99167161]])>


After 4000:
Weight: <tf.Variable 'Variable:0' shape=(1, 1) dtype=float64, numpy=array([[1.99981201]])>
Bias: <tf.Variable 'Variable:0' shape=(1, 1) dtype=float64, numpy=array([[-0.99209969]])>


After 5000:
Weight: <tf.Variable 'Variable:0' shape=(1, 1) dtype=float64, numpy=array([[1.99981195]])>
Bias: <tf.Variable 'Variable:0' shape=(1, 1) dtype=float64, numpy=array([[-0.99248704]])>


After 6000:
Weight: <tf.V

In [185]:
W

<tf.Variable 'Variable:0' shape=(1, 1) dtype=float64, numpy=array([[1.99981173]])>

In [186]:
b

<tf.Variable 'Variable:0' shape=(1, 1) dtype=float64, numpy=array([[-0.99393642]])>

In [191]:
yhat = np.squeeze(predict(X).numpy())

In [200]:
fig1 = px.line(x = xs, y = ys)
fig1.update_traces(line = dict(color = "rgb(100, 2, 2, 0.1)"))
# fig1.show()

fig2 = px.scatter(x = xs, y = yhat)

fig = go.Figure(data = fig1.data + fig2.data)
fig.show()